In [1]:
import hashlib,json, re, os
import pandas as pd
import numpy as np

from joblib import Parallel, delayed
from tqdm import tqdm

from IPython.display import Image
from IPython.core.debugger import set_trace

PATH = 'data'
# PATH = '/content/drive/MyDrive/sber_TZ/csv_data'

In [2]:
Image(url='https://static2.srcdn.com/wordpress/wp-content/uploads/2019/02/Disney-Villains-Promotional-Featured-Image.jpg?q=50&fit=crop&w=960&h=500')

### Loads text from file (text generate by lorem_ipsum)

In [3]:

line = []
with open(os.path.join(PATH, 'lorem_ipsum.txt'), 'r') as f:
    lines = f.readlines()
    print(len(lines))

# make all letter lower
line = [txt.strip().lower() for txt in lines]

250


In [4]:
line = line * 1000#250000

### Load psedo labels

In [5]:
evil_disney = [
            ['Верховный-бог Аид Геркулес','Загробный Мир 1001', 'М'],
            ['Древнегреческий-бог Аид Геркулес','Загробный Мир Тисовая 1101', 'М'],
            ['Красавчег-бог Аид Геркулес','Загробный Мир Ужасный Перекресток 666', 'М'],
            ['Билл Сайкс Оливер_и_компания', 'Приключения Оливера Твиста 19а', 'М'],
            ['Злой Гастон Красавица_и_Чудовище', 'Дремучий Лес 333', 'М'],
            ['Губернатор Рэтклифф Покахонтас', 'Королевство Англия Виргинии 1000a', 'М'],
            ['Султан Джафар Аладдин', 'Антуан Галлану 1646/1715', 'М'],
            ['Доктор Фасилье Принцесса_и_лягушка', 'Новый Орлеан 49-й', 'М'],
            ['Злая Королева Белоснежка', 'Братья Гримм 1937а', 'Ж'],
            ['Хитрая Королева Белоснежка', 'Ласке Братья Гримм 1936', 'Ж'],
            ['Восхитительная Королева Белоснежка', 'Альбертом Хуртером Братья Гримм 1937', 'Ж'],
            ['Изма Похождения Императора', 'Горы Кускотопия 1800', 'Ж'],    
            ['Капитан Крюк Питер_Пэн', 'Кенсингтонские Cады Лондон 1313', 'М'],
            ['Леди Тремейн Золушка', 'Франция Большой Замок 1812', 'Ж'],
            ['Мадам Медуза Спасатели', 'Марджери Шарп 1977', 'Ж'],
            ['Малефисента Спящая Красавица','Шарль Перро 1697', 'Ж'],
            ['Матушка Готель Рапунцель','Средневековье Башня_дом 555', 'Ж'],
            ['Персиваль Маклич Спасатели', 'В_прериях Австралии 1990', 'М'],
            ['Принц-Джон Робин Гуд', 'Шервудский Лес Ноттингем 1912', 'М'],
            ['Серый Волк Три_поросёнка', 'Англия Лондон 1843', 'М'],
            ['Стервелла Де Виль','Улица Далматинцев 101', 'Ж'],
            ['Злобная Урсула Русалочка', 'Атлантика Большой Риф 777', 'Ж'],
            ['Клод Фролло Горбун_из_Нотр-Дама', 'Франция Нотер-Дам 362', 'М'],
            ['Шерхан Книга Джунглей', 'Индия Джунгли 1893', 'М'],
            ['Шрам Король Лев', 'Африка Земели Прайда 1003', 'М'],
            ['Червонная Королева Алиса_в_Стране-чудес', 'Льюис Кэрролл 1865', 'Ж'],
            ['Эдгар Бальтазар Коты-аристократы', 'Англия Париж 1910', 'М']
            ]
df_evil = pd.DataFrame(evil_disney, columns=['name', 'address', 'sex'])
df_evil['sex'] = df_evil.sex.map({'Ж': 1, 'М': 0})
df_evil['count_name'] = df_evil.name.apply(lambda x: len(x.split()))
df_evil['count_addr'] = df_evil.address.apply(lambda x: len(x.split()))
df_evil['address'] = df_evil['address'].str.strip()
df_evil['name'] = df_evil['name'].str.strip()
df_evil.head()

,name,address,sex,count_name,count_addr
0,Верховный-бог Аид Геркулес,Загробный Мир 1001,0,3,3
1,Древнегреческий-бог Аид Геркулес,Загробный Мир Тисовая 1101,0,3,4
2,Красавчег-бог Аид Геркулес,Загробный Мир Ужасный Перекресток 666,0,3,5
3,Билл Сайкс Оливер_и_компания,Приключения Оливера Твиста 19а,0,3,4
4,Злой Гастон Красавица_и_Чудовище,Дремучий Лес 333,0,3,3


### Load data from [RuCode «Газпромбанк»](https://mipt.ru/news/partner_festivalya_rucode_gazprombank_priglashaet_reshit_realnye_zadachi_dlya_biznesa)


In [6]:
name = np.load(os.path.join(PATH, 'full_name.npy'), allow_pickle=True)
addr = np.load(os.path.join(PATH, 'addr.npy'), allow_pickle=True)


df = pd.DataFrame(name[:1142], columns = ['name'])
df['address'] = addr
bad = 'Республика Адыгея (Ад Абрикосовая 7'
df.drop(df.index[df[df.address == bad].index.values[0]], inplace=True)

df['address'] = df['address'].str.split().apply(lambda x: ' '.join(x)) 
df['name'] = df['name'].str.split().apply(lambda x: ' '.join(x)) 
df['count_name'] = df.name.apply(lambda x: len(x.split()))
df['count_addr'] = df.address.apply(lambda x: len(x.split()))

In [7]:
regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')      
for t in df.address.values:
    if(regex.search(t) != None):
        print(t)

Москва Андрухаева 74/39
Москва Андрухаева 74/39
Санкт-Петербург Майский 7/4
Московская область Братьев Лоскутовых 8/19
Москва Клубная 8/1
Краснодарский край Комсомольская 2-я 7/0
Астраханская область Дружбы 64/39
Москва Клубная 8/1


In [24]:
# """
# https://planetcalc.ru/8678/
# names.txt 2000 woman names

# find ends sur, mid, name after filter man or woman
# """

# with open(os.path.join(PATH, 'woman_names.txt'), 'r') as f:
#     lines_woman = f.readlines()
#     print(len(lines_woman))

# pd.Series(lines_woman).str.strip().str.split().apply(lambda x: x[1][-2:]).value_counts().index

### Generate data

In [9]:
def generate_key(text: str, addr:list, full_name:list)-> str:
    """
    text: str line text
    addr: list address
    full_name: names

    we take txt line and random take index position in current line and
    set name, address and inn  

    ИНН физического лица является последовательностью из 12 цифр,
    из которых первые четыре цифры — код налогового органа, который присвоил ИНН  

    return line with random name, inn and address
    """
    inn = str(np.random.randint(123456789012, 999999999999)) 
    address = np.random.choice(addr)
    full_name = np.random.choice(full_name)    
    txt = text.strip().split(' ')
    
    """
    when we generate more data we get eq index(name idx = inn idx)    
    this idea generate not merged index    
    """
    q1, q2, q3 = np.quantile([i for i in range(len(txt))], [np.random.uniform(.1,.3),
                                                            np.random.uniform(.41,.6),
                                                            np.random.uniform(.71,.9)]) 
    txt[int(q1)] = full_name
    txt[int(q2)] = inn
    txt[int(q3)] = address
    return txt

def make_psedo(df: pd.DataFrame, original_data: list, verbose = False)->tuple:
    """
    df: data PSEDO LABEL
    original_data: ['Фролов Ефрем Федотович', 'Краснодар Автодорога 7']  
    verbose: print psedo data  

    idea:
        we have data with original names
        original_data [
                       ('Третьякова', 'Михаил', 'Валерьяновина')
                       ('Москва', 'Т.Керашева', '75')
                       ]
        find len full name and address (('Третьякова', 'Михаил', 'Валерьяновина') == 3)
        after we get filter data for len
        get df for man and woman

        check simple idea find man or woman

        if all true random init index from woman df and take psedo name, address
        else man

    return: tuple(psedo name, psedo address)

    example:
        name = {'22_23_24': ('Третьякова', 'Михаил', 'Валерьяновина')}
        addr = {'68_69_70': ('Москва', 'Т.Керашева', 'Т.Керашева','Т.Керашева', '75')}
        
        name_to_gen_psedo = list(name.values())[0]
        addr_to_gen_psedo = list(addr.values())[0]
        
        original_data = [name_to_gen_psedo, addr_to_gen_psedo]

        >>  [('Третьякова', 'Михаил', 'Валерьяновина'),
            ('Москва', 'Т.Керашева', 'Т.Керашева', 'Т.Керашева', '75')]

        make_psedo(df_evil, original_data)
        >>  ('Восхитительная Королева Белоснежка', 'Альбертом Хуртером Братья Гримм 1937')

    """

    sur_bool = False
    mid_bool = False
    name_bool = False

    name, addr = original_data
    ori_sur, ori_name, ori_mid = name # ['Вишнякова', 'Хельга', 'Филатовна']
    len_name = len(name)
    len_addr = len(addr)
    df = df[(df.count_name == len_name) & (df.count_addr == len_addr)]
   
    woman = df[df.sex == 1].reset_index(drop = True)
    man = df[df.sex != 1].reset_index(drop =True)
   
    # take sex
    if ori_sur.endswith(('ва', 'на', 'ая', 'ха')):
        sur_bool =True 
    if ori_mid.endswith('на'):
        mid_bool =True
    if ori_name.endswith(('ия', 'на', 'ра', 'са', 'та', 'ва', 'ка', 'ья', 'га', 'йя', 'да', 'ма', 'оя', 'ла', 'ся', 'вь')):
        name_bool =True
    
    if (name_bool == True) & (sur_bool == True) & (mid_bool == True): # woman     
        idx = np.random.randint(woman.shape[0])
        psedo_name, psedo_address = woman.iloc[idx].values[:2]        
    else:       
        idx = np.random.randint(man.shape[0])
        psedo_name, psedo_address = man.iloc[idx].values[:2] 

    if verbose:
        print('Fake data', psedo_name, psedo_address)    
    return (psedo_name, psedo_address)

In [10]:
# generate name, address and inn for blank text
namev = df.name.values
addrev = df.address.values

tmp = []
for i in range(len(line)):
    d = generate_key(line[i], addrev, namev)
    tmp.append(' '.join(d).strip())

print('Len names: ', len(np.unique(name)),
      'len unq. address: ',len(np.unique(addr)),
      'len address from .csv: ', len(addr),
      'len generate data: ', len(tmp))

Len names:  51529 len unq. address:  254 len address from .csv:  1142 len generate data:  250000


### Search and depersonalization of customer data in unstructured text

In [11]:
"""
In line only one (name, address, inn)
"""

def find_address(txt: str)-> dict:
    """
    txt: str line text
    find index digit in line

    we get index:
        min len address is 3
        we check:            
            (data[i-2][0].isupper())<---data[i-1][0].isupper())<---75
            get ('Москва', 'Т.Керашева', '75')

    return max len(index) because we get for len 3 len 4
    example:
            return (
                    {'68_69_70': ('Москва', 'Т.Керашева', '75')
                    {'68_69_70_71': ('Москва', 'Т.Керашева', 'Лоботрясовая', '75')
                    )
            sorted(tmp)[0] -> 68_69_70_71

    return index words in address : list: address {'68_69_70_71': ('Москва', 'Т.Керашева', 'Лоботрясовая', '75')}
    """
    tmp = {}
    count = []
    clear_txt = ' '.join(txt.split()) 
    data = clear_txt.split(' ') 
    # find index digits
    for idx, w in enumerate(data):
        if w[0].isdigit():
            count.append(idx)
 
    for i in count:
        #  Москва Дубзаводская 147  
        if (data[i-1][0].isupper()) and (data[i-2][0].isupper()):
            txt_addr = data[i-2], data[i-1], data[i]            
            tmp[f'{i-2}_{i-1}_{i}'] = txt_addr
            # Дубзаводская 147 08 
            if (data[i+1][0].isdigit()) and (len(data[i+1]) < 12):
                txt_addr = data[i-2], data[i-1], data[i], data[i+1]
                tmp[f'{i-2}_{i-1}_{i}_{i+1}'] = txt_addr
        # Краснодарский край Автодорога 7 
        if (data[i-1][0].isupper()) and (data[i-2][0].islower()) and (data[i-3][0].isupper()):
            txt_addr = data[i-3], data[i-2], data[i-1], data[i]
            tmp[f'{i-3}_{i-2}_{i-1}_{i}'] = txt_addr
            # Автодорога 7 3А 
            if (data[i+1][0].isdigit()) and (len(data[i+1]) < 12):
                txt_addr = data[i-3], data[i-2], data[i-1], data[i], data[i+1]
                tmp[f'{i-3}_{i-2}_{i-1}_{i}_{i+1}'] = txt_addr
        # Москва Т.Керашева Лоботрясовая 75
        if (data[i-1][0].isupper()) and (data[i-2][0].isupper()) and (data[i-3][0].isupper()):
            txt_addr = data[i-3], data[i-2], data[i-1], data[i]
            tmp[f'{i-3}_{i-2}_{i-1}_{i}'] = txt_addr
            if (data[i+1][0].isdigit()) and (len(data[i+1]) < 12):
                txt_addr = data[i-3], data[i-2], data[i-1], data[i], data[i+1]
                tmp[f'{i-3}_{i-2}_{i-1}_{i}_{i+1}'] = txt_addr
        # Моск. Область Т.Керашева Лоботрясовая 75
        if (data[i-1][0].isupper()) and (data[i-2][0].isupper()) and (data[i-3][0].isupper()) and (data[i-4][0].isupper()):
            txt_addr = data[i-4], data[i-3], data[i-2], data[i-1], data[i]
            tmp[f'{i-4}_{i-3}_{i-2}_{i-1}_{i}'] = txt_addr
            if (data[i+1][0].isdigit()) and (len(data[i+1]) < 12):
                txt_addr = data[i-4], data[i-3], data[i-2], data[i-1], data[i], data[i+1]
                tmp[f'{i-4}_{i-3}_{i-2}_{i-1}_{i}_{i+1}'] = txt_addr


    if len(tmp) > 1:  return {sorted(tmp, key=len, reverse=True)[0] :tmp[sorted(tmp, key=len, reverse=True)[0]]}
    else: return tmp

def find_full_name(txt: str)->dict:  
    """
    txt: str line text    
    return {'22_23_24': ('Третьяков', 'Михаил', 'Валерьянович')}
    similarly how find_address but we find upper latter
    """  
    tmp = {}
    count = []
    clear_txt = ' '.join(txt.split()) 
    data = clear_txt.split(' ')
    for idx, w in enumerate(data):
        if w[0].isupper():
            count.append(idx)    
    for i in range(len(count)):
        if data[count[i] + 2][0].isupper():       
            if data[count[i]+3][0].isupper():
                # for another dataset with name  len > 3
                # if data[count[i ] + 1][0].isupper():   
                #     txt_name = data[count[i]], data[count[i]+ 1], data[count[i ]+ 2],data[count[i ]+ 3]
                #     tmp[f'{count[i]}_{count[i]+1}_{count[i]+2}_{count[i]+3}'] = txt_name
                pass                    
            else:
                if data[count[i ] + 1][0].isupper():
                    txt_name = data[count[i]], data[count[i + 1]], data[count[i + 2]]
                    tmp[f'{count[i]}_{count[i]+1}_{count[i]+2}'] = txt_name
    # not need all data len names is 3
    return tmp

In [12]:
def take_txt(txt: str, df: pd.DataFrame, decode_data:dict = None, key: str = None)-> str:
    """
    txt:srt line text with added mark(name, address and inn)
    df: psedo df 
    key: str = None df96220fa161767c5cbb95567855c86b
    Example:
        masked:
            find name  ---> {'22_23_24': ('Третьякова', 'Михаил', 'Валерьяновина')}
            and address ---> {'68_69_70': ('Москва', 'Т.Керашева', 'Т.Керашева', 'Т.Керашева', '75')}
            decode is None:
                find inn ---> '123456789123'
                get hex ---> 'df96220fa161767c5cbb95567855c86b'
                get original data:
                    ('Третьяков', 'Михаил', 'Валерьянович')
                    ('Москва', 'Москва', 'Т.Керашева', 'Т.Керашева', '75')
                psedo:
                    name_psedo --->  Красавчег_бог Аид Геркулес'
                    addr_psedo ---> 'Загробный Мир Ужасный Перекресток 666'
                save to decode after
            get name index to set psedo:
                '22_23_24' ---> ['22', '23', '24']
            split line
            set psedo by index and join text for line after change original inn to hex
        decode:
            we get data from previos saved in masked save_to_decode
            name_psedo ---> {'26_27_28': ('Голыгиев', 'Ландри', 'Всеволодович')}
            addr_psedo ---> {'80_81_82': ('Москва', 'Вишневая', '70')}
            inn ---> 269054776996
            after split:
                        name_psedo -->  ('Голыгиев', 'Ландри', 'Всеволодович')
            other step indentical masked    
    if decode return txt
    if masked return decode_data and txt
        decode_data:
            {'df96220fa161767c5cbb95567855c86b': [{'68_69_70': ('Москва','Т.Керашева','75')},
                                                {'22_23_24': ('Третьяков', 'Михаил', 'Валерьянович')},
                                                '123456789123']}
    """
    # set_trace()
    name = find_full_name(txt)
    addr = find_address(txt)

    if decode_data is None:          
        save_to_decode = {}        
        reg_inn = re.search('\d{12}', txt).group()
        key_hex = hashlib.md5(reg_inn.encode('utf-8')).hexdigest() 

        name_to_gen_psedo = list(name.values())[0]
        addr_to_gen_psedo = list(addr.values())[0]
        original_data = [name_to_gen_psedo, addr_to_gen_psedo]
        # name_psedo, addr_psedo
        name_psedo, addr_psedo = make_psedo(df, original_data)
        name_psedo, addr_psedo = name_psedo.split(), addr_psedo.split()

        save_to_decode[key_hex] = [name, addr, reg_inn]
    else:     
        name_psedo, addr_psedo, reg_inn = decode_data[key]
        name_psedo, addr_psedo = name_psedo[list(name.keys())[0]], addr_psedo[list(addr.keys())[0]]

    # change to psedo    
    idx_name = list(name.keys())[0].split('_')
    assert len(idx_name) == 3, f'error len name < 3 {list(name.values())[0]}'
    idx_addr = list(addr.keys())[0].split('_')
    # count all len addr in data set and set to assert < max values len addr
    assert len(idx_addr) >= 3, f'error len idx_addr >= 3 {list(idx_addr.values())[0]}' 
    data = txt.split()
    psedo_data = data.copy()  
    psedo_data[int(idx_name[0]): int(idx_name[-1])+1] = name_psedo
    psedo_data[int(idx_addr[0]): int(idx_addr[-1])+1] = addr_psedo
    psedo_idx = (np.array(data) != np.array(psedo_data)).nonzero()[0]
    psedo_idx_name = psedo_idx[:3]
    psedo_idx_addr = psedo_idx[3:]       

    assert all(psedo_idx_name == np.asarray(idx_name).astype(int)),\
           f'idx name not corect {idx_name} -> {psedo_idx_name}'
    assert all(psedo_idx_addr == np.asarray(idx_addr).astype(int)),\
               f'idx addr not corect {idx_addr} -> {psedo_idx_addr}'
    new_txt = ' '.join(psedo_data)    
    if decode_data is None:             
        new_txt = re.sub(reg_inn, key_hex, new_txt)
        return new_txt, save_to_decode
    else: return re.sub(key, reg_inn, new_txt)

In [13]:
new_txt, param = take_txt(tmp[0], df_evil)
decode_txt = take_txt(new_txt, df_evil, decode_data=param, key=list(param.keys())[0])
assert decode_txt == tmp[0], 'simple check decode not correct'
print('Correct!')

Correct!


In [14]:
# !cat /proc/cpuinfo

In [15]:
# %%time
# ori_data = {}
# fake_txt = []
# for text in tmp:
#     new_txt, param = take_txt(text, df_evil)    
#     ori_data.update(param)
#     fake_txt.append(new_txt)

In [16]:
# %%time
# make_ori_txt = []
# for key in ori_data.keys():
#     for t in fake_txt:
#         if key in t:   
#             ori_txt =  take_txt(t, df_evil, decode_data=ori_data, key = key)
#             make_ori_txt.append(ori_txt)

In [17]:
# for i, j in zip(make_ori_txt, tmp):
#     if i != j:
#         print(i)
#         print(j)

In [18]:
%%time
result = Parallel(n_jobs = -1)(delayed(take_txt)(text,
                                        df_evil              
                                            )
                                            for text in tqdm(tmp)
                                            )

100%|██████████| 250000/250000 [01:07<00:00, 3730.91it/s]


CPU times: user 20.8 s, sys: 1.79 s, total: 22.6 s
Wall time: 1min 7s


In [19]:
%%time
decode_txt = Parallel(n_jobs = -1)(delayed(take_txt)(result[idx][0],
                                                     df_evil,
                                                     result[idx][1],
                                                     list(result[idx][1].keys())[0]
                                                     ) 
                                                     for idx in range(len(result))
                                                     )

CPU times: user 7.2 s, sys: 629 ms, total: 7.83 s
Wall time: 8.57 s


### Check correct

In [20]:
result[127][0]

'повседневная практика показывает, что консультация с широким активом позволяет выполнять важные задания по разработке позиций, занимаемых участниками в отношении поставленных Капитан Крюк Питер_Пэн товарищи! реализация намеченных плановых заданий способствует подготовки и реализации форм развития. не следует, однако забывать, что начало повседневной работы по формированию позиции в значительной степени обуславливает создание систем массового участия. значимость этих проблем настолько очевидна, что 15a05e431996d6bb78c8372bcc043df1 повседневной работы по формированию позиции требуют от нас анализа соответствующий условий активизации. не следует, однако забывать, что реализация намеченных плановых заданий позволяет оценить значение направлений Кенсингтонские Cады Лондон 1313 развития. не следует, однако забывать, что начало повседневной работы по формированию позиции требуют от нас анализа модели развития.'

In [21]:
result[127][1]

{'15a05e431996d6bb78c8372bcc043df1': [{'20_21_22': ('Близнякова',
    'Роини',
    'Будимировна')},
  {'85_86_87_88': ('Москва', 'Комсомольская', '2-я', '6')},
  '476592468897']}

In [22]:
decode_txt[127]

'повседневная практика показывает, что консультация с широким активом позволяет выполнять важные задания по разработке позиций, занимаемых участниками в отношении поставленных Близнякова Роини Будимировна товарищи! реализация намеченных плановых заданий способствует подготовки и реализации форм развития. не следует, однако забывать, что начало повседневной работы по формированию позиции в значительной степени обуславливает создание систем массового участия. значимость этих проблем настолько очевидна, что 476592468897 повседневной работы по формированию позиции требуют от нас анализа соответствующий условий активизации. не следует, однако забывать, что реализация намеченных плановых заданий позволяет оценить значение направлений Москва Комсомольская 2-я 6 развития. не следует, однако забывать, что начало повседневной работы по формированию позиции требуют от нас анализа модели развития.'

In [23]:
assert decode_txt == tmp, 'not eq, decode error'
print('Correct!')

Correct!
